# Capstone Project - The Battle of Neighborhoods- FINAL - M Gogoi

# Introduction

### __Background__

The Economist states that the London Housing Market is taking a downward spiral. It is now facing several different headwinds, including the outlook of higher taxes and a dire warning from the Bank of England that U.K. home values could fall as much as 30% in the occasion of a muddled exodus from the European Union. More precisely, four overlooked cracks advocate that the London market may be in worst shape than many apprehend: hidden price falls, record-low sales, homebuilder emigration and tax hikes addressing overseas purchasers of households in England and Wales.

### __Business Problem__

In this situation, it is crucial to implement machine learning tools in order to support homebuyer clienteles in London to make intelligent and effective choices and decisions. As a result of this, the business problem we are now proposing is: how could we deliver support to homebuyers clientele in London to purchase an appropriate real estate in this times of ambiguous economic and financial situations? To solve this business problem, we are going to cluster London neighborhoods in order to recommend locations and the current average price of real estate where homebuyers can make a real estate investment. We will recommend moneymaking venues according to facilities and essential amenities surrounding such locations i.e. elementary schools, high schools, hospitals, grocery stores, fitness centers, parks etc.

## __Data section__

Data on London properties and the relative price paid statistics were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise of the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically, the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County. To discover and target recommended sites across different locations according to the presence of facilities and vital amenities, we will access data through FourSquare API interface and arrange them as a data frame for imagining. By amalgamation of data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such chattels from FourSquare API interface, we will be able to endorse profitable real estate investments.

## __Methodology__

The Methodology section comprises of the main components of our analysis and predictive mechanism. The Methodology section comprises of four stages:

1. Collecting Data for inspection
2. Exploring and Understanding the Data
3. Data preparation and pre-processing 
4. Modeling

__1. Collecting Data for inspection__

After importing the necessary libraries, we downloaded the data from the HM Land Registry website:


In [4]:
import os 
import numpy as np
import pandas as pd
import datetime as dt 
import json 

In [5]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [6]:
import requests 
from pandas.io.json import json_normalize 

In [7]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [8]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [9]:
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

__2. Explore and Understand Data__

We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [10]:
df_ppd.head(5)

,{79A74E21-D11E-1289-E053-6B04A8C01627},770000,2018-09-25 00:00,SK7 1AR,D,N,F,5,Unnamed: 8,OAK MEADOW,BRAMHALL,STOCKPORT,STOCKPORT.1,GREATER MANCHESTER,A,A.1
0,{79A74E21-D11F-1289-E053-6B04A8C01627},253500,2018-09-24 00:00,M6 8GQ,D,N,F,1,NaN,RIVINGTON ROAD,NaN,SALFORD,SALFORD,GREATER MANCHESTER,A,A
1,{79A74E21-D120-1289-E053-6B04A8C01627},231950,2018-09-28 00:00,WA3 2UE,D,Y,F,35,NaN,STONEACRE CLOSE,LOWTON,WARRINGTON,WIGAN,GREATER MANCHESTER,A,A
2,{79A74E21-D121-1289-E053-6B04A8C01627},112500,2018-08-29 00:00,OL6 6RJ,S,N,F,102,NaN,THORNFIELD GROVE,NaN,ASHTON-UNDER-LYNE,TAMESIDE,GREATER MANCHESTER,A,A
3,{79A74E21-D122-1289-E053-6B04A8C01627},184995,2018-06-15 00:00,M46 0TW,S,Y,F,37,NaN,THREADNEEDLE PLACE,ATHERTON,MANCHESTER,WIGAN,GREATER MANCHESTER,A,A
4,{79A74E21-D123-1289-E053-6B04A8C01627},214995,2018-09-28 00:00,M28 3XS,D,Y,L,9,NaN,MARPLE GARDENS,WORSLEY,MANCHESTER,SALFORD,GREATER MANCHESTER,A,A


In [11]:
df_ppd.shape

(1023149, 16)

__3. Data preparation and preprocessing__

At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

1. Rename the column names

2. Format the date column

3. Sort data by date of sale

4. Select data only for the city of London

5. Make a list of street names in London

6. Calculate the street-wise average price of the property

7. Read the street-wise coordinates into a dataframe,eliminating recurring word London from individual names

8. Merge the data to find the coordinates of locations which fit into client's budget

9. Plot recommended locations on London map along with current market prices

In [12]:
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [14]:
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [15]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")
streets = df_ppd_london['Street'].unique().tolist()

In [16]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()
df_grp_price.columns = ['Street', 'Avg_Price']

In [17]:
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2.450000e+06
391,ANHALT ROAD,2.435000e+06
406,ANSDELL TERRACE,2.250000e+06
421,APPLEGARTH ROAD,2.400000e+06
700,AYLESTONE AVENUE,2.286667e+06
...,...,...
13691,WILFRED STREET,2.410538e+06
13717,WILLOW BRIDGE ROAD,2.425000e+06
13737,WILSON STREET,2.257500e+06
13765,WINCHENDON ROAD,2.350000e+06


In [18]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty

from sklearn.cluster import KMeans

In [19]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 391
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 391, dtype: object
item.Street only: ANHALT ROAD
index: 406
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 406, dtype: object
item.Street only: ANSDELL TERRACE
index: 421
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 421, dtype: object
item.Street only: APPLEGARTH ROAD
index: 700
item: Street       AYLESTONE AVENUE
Avg_Price         2.28667e+06
Name: 700, dtype: object
item.Street only: AYLESTONE AVENUE
index: 854
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 854, dtype: object
item.Street only: BARONSMEAD ROAD
index: 980
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 980, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1101
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name

In [20]:
geolocator = Nominatim()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [21]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
391,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
406,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
421,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)"
700,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)"
...,...,...,...
13691,WILFRED STREET,2.410538e+06,"(53.5116186, -2.1947278)"
13717,WILLOW BRIDGE ROAD,2.425000e+06,"(51.5431088, -0.0955079)"
13737,WILSON STREET,2.257500e+06,"(38.4394642, -122.7221956)"
13765,WINCHENDON ROAD,2.350000e+06,"(51.7955877, -0.9639361)"


In [28]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
391,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
406,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
421,APPLEGARTH ROAD,2.400000e+06,"(53.749244, -0.32678)",53.749244,-0.326780
700,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)",51.540916,-0.217874
...,...,...,...,...,...
13691,WILFRED STREET,2.410538e+06,"(53.5116186, -2.1947278)",53.511619,-2.194728
13717,WILLOW BRIDGE ROAD,2.425000e+06,"(51.5431088, -0.0955079)",51.543109,-0.095508
13737,WILSON STREET,2.257500e+06,"(38.4394642, -122.7221956)",38.439464,-122.722196
13765,WINCHENDON ROAD,2.350000e+06,"(51.7955877, -0.9639361)",51.795588,-0.963936


In [30]:
df = df_affordable.drop(columns=['city_coord'])

In [31]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
391,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
406,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
421,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
700,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874
...,...,...,...,...
13691,WILFRED STREET,2.410538e+06,53.511619,-2.194728
13717,WILLOW BRIDGE ROAD,2.425000e+06,51.543109,-0.095508
13737,WILSON STREET,2.257500e+06,38.439464,-122.722196
13765,WINCHENDON ROAD,2.350000e+06,51.795588,-0.963936


In [32]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.4893335, -0.144055084527687.


In [33]:
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [34]:
CLIENT_ID = '3EI0RVQAPVSNF5JB1NJAJQZNRMV3FCU2QMI5BY4NOCXJ4PFJ' # Foursquare ID
CLIENT_SECRET = 'BBDAFESWPHRAHWURYKOHXAJ2IWVT4OGC2GMDLSUUPQGYVVMY' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 3EI0RVQAPVSNF5JB1NJAJQZNRMV3FCU2QMI5BY4NOCXJ4PFJ
CLIENT_SECRET:BBDAFESWPHRAHWURYKOHXAJ2IWVT4OGC2GMDLSUUPQGYVVMY


__4. Modeling__

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.


In [35]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [36]:
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude'])

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
AYLESTONE AVENUE
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENSI

In [37]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
3,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
4,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
...,...,...,...,...,...,...,...
5815,WINGATE ROAD,57.168256,-2.108925,tiki cafe,57.166838,-2.104161,Café
5816,WINGATE ROAD,57.168256,-2.108925,St Machar's Cathedral,57.169326,-2.102846,Church
5817,WINGATE ROAD,57.168256,-2.108925,JG Ross,57.165434,-2.104557,Bakery
5818,WINGATE ROAD,57.168256,-2.108925,King's Museum,57.166757,-2.102294,History Museum


In [38]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,26,26,26,26,26,26
ANHALT ROAD,14,14,14,14,14,14
ANSDELL TERRACE,59,59,59,59,59,59
APPLEGARTH ROAD,4,4,4,4,4,4
AYLESTONE AVENUE,5,5,5,5,5,5
...,...,...,...,...,...,...
WHITFIELD STREET,18,18,18,18,18,18
WILFRED STREET,5,5,5,5,5,5
WILLOW BRIDGE ROAD,25,25,25,25,25,25


In [39]:
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 353 uniques categories.


In [40]:
location_venues.shape

(5820, 7)

In [41]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBION SQUARE,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0
1,ANHALT ROAD,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0
2,ANSDELL TERRACE,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.016949,0.0,0.0
3,APPLEGARTH ROAD,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0
4,AYLESTONE AVENUE,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,WHITFIELD STREET,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0
147,WILFRED STREET,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0
148,WILLOW BRIDGE ROAD,0.0,0.000,0.0,0.0,0.0,0.00,0.000,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.000000,0.0,0.0
149,WILSON STREET,0.0,0.025,0.0,0.0,0.0,0.05,0.025,0.0,0.0,...,0.0,0.0,0.0,0.025,0.0,0.0,0.0,0.000000,0.0,0.0


In [43]:
london_grouped.shape

(151, 354)

In [44]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
         venue  freq
0         Café  0.19
1   Restaurant  0.08
2          Bar  0.08
3  Coffee Shop  0.08
4          Pub  0.08


----ANHALT ROAD----
                 venue  freq
0                  Pub  0.29
1        Grocery Store  0.14
2   English Restaurant  0.07
3  Japanese Restaurant  0.07
4          Pizza Place  0.07


----ANSDELL TERRACE----
                venue  freq
0          Restaurant  0.07
1      Clothing Store  0.07
2  Italian Restaurant  0.05
3                 Pub  0.05
4                Café  0.05


----APPLEGARTH ROAD----
               venue  freq
0                Pub  0.50
1          Nightclub  0.25
2             Casino  0.25
3                ATM  0.00
4  Paella Restaurant  0.00


----AYLESTONE AVENUE----
           venue  freq
0           Park   0.4
1       Bus Stop   0.2
2           Café   0.2
3  Movie Theater   0.2
4            ATM   0.0


----BARONSMEAD ROAD----
                 venue  freq
0    Food & Drink Shop  0.15
1        Movie Theater  0

In [45]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [46]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [47]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [48]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Pub,Bar,Indian Restaurant,Coffee Shop,Restaurant,New American Restaurant,French Restaurant,Fish & Chips Shop,Supermarket
1,ANHALT ROAD,Pub,Grocery Store,Japanese Restaurant,Plaza,Gym / Fitness Center,English Restaurant,Cocktail Bar,Pizza Place,Garden,Diner
2,ANSDELL TERRACE,Restaurant,Clothing Store,Café,Italian Restaurant,Hotel,Pub,Juice Bar,Chinese Restaurant,English Restaurant,Bakery
3,APPLEGARTH ROAD,Pub,Nightclub,Casino,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
4,AYLESTONE AVENUE,Park,Movie Theater,Bus Stop,Café,Food Stand,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food


In [49]:
venues_sorted.shape

(151, 11)

In [50]:
london_grouped.shape

(151, 354)

In [51]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [52]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([0, 3, 1, 3, 4, 2, 0, 2, 1, 1, 0, 3, 3, 0, 0, 2, 2, 0, 3, 1, 0, 4,
       4, 3, 0, 0, 1, 3, 4, 0, 1, 3, 2, 2, 2, 4, 3, 3, 2, 1, 0, 2, 4, 1,
       4, 1, 4, 1, 1, 3], dtype=int32)

In [53]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
391,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
406,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
421,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780
700,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874


In [54]:
london_grouped_clustering.shape

(161, 4)

In [55]:
df.shape

(161, 4)

In [56]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [57]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [58]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,0,Café,Pub,Bar,Indian Restaurant,Coffee Shop,Restaurant,New American Restaurant,French Restaurant,Fish & Chips Shop,Supermarket
391,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,3,Pub,Grocery Store,Japanese Restaurant,Plaza,Gym / Fitness Center,English Restaurant,Cocktail Bar,Pizza Place,Garden,Diner
406,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,1,Restaurant,Clothing Store,Café,Italian Restaurant,Hotel,Pub,Juice Bar,Chinese Restaurant,English Restaurant,Bakery
421,APPLEGARTH ROAD,2.400000e+06,53.749244,-0.326780,3,Pub,Nightclub,Casino,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
700,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874,4,Park,Movie Theater,Bus Stop,Café,Food Stand,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
854,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Food & Drink Shop,Nature Preserve,Pizza Place,Pub,Breakfast Spot,Movie Theater,Park,Coffee Shop,Community Center,Thai Restaurant
980,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,0,Pub,Coffee Shop,Hotel,Grocery Store,Thai Restaurant,Bakery,Cocktail Bar,Falafel Restaurant,Chinese Restaurant,Fast Food Restaurant
1101,BELVEDERE DRIVE,2.340000e+06,44.762842,-63.669231,2,Gas Station,Sporting Goods Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
1214,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934,1,Hotel,Café,Pizza Place,Coffee Shop,Gastropub,Indian Restaurant,Italian Restaurant,Chinese Restaurant,Restaurant,Burger Joint
1252,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,1,Pub,Train Station,Brewery,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Coffee Shop,Bakery,Lake,Dive Bar


In [59]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [60]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Pub,Bar,Indian Restaurant,Coffee Shop,Restaurant,New American Restaurant,French Restaurant,Fish & Chips Shop,Supermarket
980,2480000.0,Pub,Coffee Shop,Hotel,Grocery Store,Thai Restaurant,Bakery,Cocktail Bar,Falafel Restaurant,Chinese Restaurant,Fast Food Restaurant
1552,2456875.0,Men's Store,Middle Eastern Restaurant,Lake,Food Court,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
1913,2445000.0,Café,Brewery,Bakery,Diner,Pizza Place,Convenience Store,Coffee Shop,Fish & Chips Shop,Garden Center,Gas Station
1979,2492500.0,Supermarket,English Restaurant,Pub,Rental Car Location,American Restaurant,Gym,Café,Park,Zoo Exhibit,Flower Shop


In [61]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
406,2250000.0,Restaurant,Clothing Store,Café,Italian Restaurant,Hotel,Pub,Juice Bar,Chinese Restaurant,English Restaurant,Bakery
1214,2208500.0,Hotel,Café,Pizza Place,Coffee Shop,Gastropub,Indian Restaurant,Italian Restaurant,Chinese Restaurant,Restaurant,Burger Joint
1252,2217000.0,Pub,Train Station,Brewery,French Restaurant,Gym / Fitness Center,Chinese Restaurant,Coffee Shop,Bakery,Lake,Dive Bar
2225,2200000.0,Nightlife Spot,Playground,Soccer Field,Frozen Yogurt Shop,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market
2637,2250000.0,Gastropub,Construction & Landscaping,Food Court,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food


In [62]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
854,2375000.0,Food & Drink Shop,Nature Preserve,Pizza Place,Pub,Breakfast Spot,Movie Theater,Park,Coffee Shop,Community Center,Thai Restaurant
1101,2340000.0,Gas Station,Sporting Goods Shop,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
2067,2375000.0,Pub,Park,Hotel,Yoga Studio,Coffee Shop,Grocery Store,Bakery,Italian Restaurant,Indian Restaurant,Bubble Tea Shop
2128,2379652.7,Pub,Bakery,Coffee Shop,Park,Hotel,Indian Restaurant,Grocery Store,Hostel,Yoga Studio,Movie Theater
2943,2367500.0,Hotel,Pub,Café,Italian Restaurant,Garden,Coffee Shop,Mediterranean Restaurant,Chinese Restaurant,Indian Restaurant,Supermarket


In [63]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
391,2435000.0,Pub,Grocery Store,Japanese Restaurant,Plaza,Gym / Fitness Center,English Restaurant,Cocktail Bar,Pizza Place,Garden,Diner
421,2400000.0,Pub,Nightclub,Casino,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop
1631,2397132.0,Italian Restaurant,Other Great Outdoors,Coffee Shop,Indian Restaurant,Grocery Store,Athletics & Sports,Zoo Exhibit,Food & Drink Shop,Fast Food Restaurant,Filipino Restaurant
1796,2400000.0,Art Gallery,Zoo Exhibit,Food Court,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
2158,2425000.0,Coffee Shop,Hotel,Pub,Italian Restaurant,Burger Joint,Café,Bar,Cocktail Bar,History Museum,Clothing Store


In [64]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
700,2.286667e+06,Park,Movie Theater,Bus Stop,Café,Food Stand,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
2242,2.300000e+06,Campground,Zoo Exhibit,Cycle Studio,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food
2405,2.286679e+06,Café,Bar,Italian Restaurant,Coffee Shop,Pizza Place,Pub,Vegetarian / Vegan Restaurant,Market,Restaurant,Gymnastics Gym
2685,2.287500e+06,Pub,Brewery,Gift Shop,Art Museum,Gym / Fitness Center,Food Court,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market
3376,2.298000e+06,Hotel,Zoo Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop



# Results and Discussion section

First of all, even though the London Housing Market may be in a pothole, it is still an "ever-green" for business activities.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is exciting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable locations to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our outcomes according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.


# Conclusion

To sum up, according to The Economist, the London Housing Market is in a pothole. It is now facing several different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?
To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.
First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.
Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.
Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.